In [ ]:
# Importing the libraries for the layers of CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
import matplotlib.pyplot as plt
from skimage.transform import resize

In [ ]:
from keras.layers import Dense, InputLayer, BatchNormalization, Dropout
from keras.models import Sequential
# importing adam optimizer from keras optimizer module 
from keras.optimizers import Adam

# train_test_split to create training and validation set
from sklearn.model_selection import train_test_split
# accuracy_score to calculate the accuracy of predictions
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_path="dataset/dataset/train/"
real_path="training_real/"
fake_path="training_fake/"

In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

In [ ]:
training_image_list = list()

In [ ]:
for idx in train_df.file_id:
    if(idx <= 759):
        img = plt.imread(train_path+fake_path + str(idx)+".jpg")
    else:
        img = plt.imread(train_path+real_path + str(idx)+".jpg")
    resize_img = resize(img, output_shape=(175,175))
    training_image_list.append(resize_img)

In [ ]:
image_list = np.array(training_image_list)
image_list.shape

In [ ]:
#Normalization 
image_list = image_list/image_list.max()

In [ ]:
y = train_df['label']
y.shape

In [ ]:
X=image_list

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# defining the model architecture

# creating a sequential model 
model = Sequential()
    
# defining input layer with 3D input of shape (224,224,3)
model.add(InputLayer(shape=(200, 200, 3)))

# defining the first convolutional layer with 25 filters of size (5,5)
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='valid'))
# defining the second convolutional layer with 50 filters of size (5,5)
model.add(Conv2D(50, (5, 5), activation='relu', strides=(1, 1), padding='valid'))

# flattening the output from convolutional layers so that it can be forwarded to the dense layers
model.add(Flatten())

# defining the first dense or fully connected layer with 100 neurons
model.add(Dense(units=100, activation='sigmoid'))
# defining the output layer with 1 neuron since it is a binary classification problem
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(100,
                                  100,
                                  3)),
    #layers.RandomRotation(0.01),
    layers.RandomZoom(0.1),
  ])
rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
])

model = Sequential([
  data_augmentation,
  rescale,
  layers.Conv2D(60, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(125, 3, padding='same', activation='relu'),
  layers.Conv2D(125, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(250, 3, padding='same', activation='relu'),
  layers.Conv2D(250, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(250, 3, padding='same', activation='relu'),
  layers.Conv2D(250, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(300, activation='relu'),
  layers.BatchNormalization(),
  layers.Dropout(0.2),
  layers.Dense(300, activation='relu'),
  layers.BatchNormalization(),
  layers.Dropout(0.2),
  layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
# compiling the model
model.compile(loss='binary_crossentropy', optimizer="sgd", metrics=['accuracy'])

In [ ]:
# training the model
model_history = model.fit(X_train, y_train, epochs=50, batch_size=128,validation_data=(X_test,y_test))

In [ ]:
from tensorflow.keras.applications import DenseNet121

In [ ]:
dense_mod = DenseNet121(weights='imagenet', include_top=False, input_shape=(175,175,3))
for layer in dense_mod.layers[-10:]:
    layer.trainable=True

In [ ]:
x=dense_mod.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(500, activation='relu')(x)
predictions=tf.keras.layers.Dense(1, activation='sigmoid')(x)
model1=tf.keras.Model(inputs=dense_mod.input, outputs=predictions)
model1.compile(loss='binary_crossentropy', optimizer="sgd", metrics=['accuracy'])


In [ ]:
model1.summary()

In [ ]:
model1_history = model1.fit(X_train, y_train, epochs=18, batch_size=64,validation_data=(X_test,y_test))

In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

In [ ]:
test_files_path = "dataset/dataset/test/test/"

In [ ]:
test_image_list = list()

In [ ]:
for idx in test_df.file_id:
    img = plt.imread(test_files_path + str(idx)+".jpg")
    resize_img = resize(img, output_shape=(175,175))
    test_image_list.append(resize_img)

In [ ]:
test_image_list = np.array(test_image_list)
test_image_list.shape

In [ ]:
#Normalization 
test_image_list = test_image_list/test_image_list.max()

In [ ]:
output_prob = model1.predict(test_image_list)

In [ ]:
test_image_list[0].shape

In [ ]:
output_prob[31]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(test_image_list[42])
plt.show()

In [ ]:
output_df = pd.DataFrame({"file_id":test_df.file_id, "label":pd.Series()})

In [ ]:
for i in range(output_df.shape[0]):
    output_df.loc[i, "label"] = 0 if output_prob[i] <0.93 else 1

In [ ]:
output_df.head(20)

In [ ]:
output_df.to_csv("Output_3_93_per.csv", index=False)